\[<< [Sequence, Iterator and Generator](./06_sequence_iterators_and_generators.ipynb) | [Index](./00_index.ipynb) | [Building command line interface](./08_argparse.ipynb) >>\]`

# Context Managers

`with` statements in Python make sense when you think of problem they are trying to solve.

```python
set something up
try:
    do something
finally:
    tear something down
```

Here `set something up` can be opening a file, opening a database connection, acquiring some external resource and `tear something down` can be closing a file, closing a database connection or releasing an acquired resource.

In [7]:
file = open("static/file1.txt", "w+")
try:
    file.write("Hello there")
finally:
    file.close()

In [2]:
from IPython.display import Code

Code(filename="static/file1.txt")

Hello there

In [3]:
with open("static/file1.txt", "w+") as file:  # <--- Enter context: Open a file
    file.write("Hello there - again")  # <--- Do some work: Writing a file
    # <--- Exit context: Close a file

In [6]:
Code(filename="static/file1.txt")

Hello there

Context manager manages the data in a scope `on entry` and cleans it up `on exit`.

- Open/Close file
- Open/Close database
- Set decimal precision to 3/Reset it back to default

In [8]:
with open("static/file1.txt", "w+") as file1, open("static/file2.txt", "w+") as file2:
    file1.write("Hello from file1")
    file2.write("Hello from file2")

In [9]:
Code(filename="static/file1.txt")

Hello from file1

In [10]:
Code(filename="static/file2.txt")

Hello from file2

In [11]:
with open("static/file1.txt", "w+") as file1:
    with open("static/file2.txt", "w+") as file2:
        file1.write("Hello from file1")
        file2.write("Hello from file2")

## Writing context manager the hard way (using class)

In [12]:
class CtxMng:
    def __init__(self):
        print("inside __init__")

    def __enter__(self):
        print("inside __enter__")

        return (1, 2, 3)

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("inside __exit__")

In [13]:
with CtxMng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")

inside __init__
inside __enter__
x = 1, y = 2, z = 3
inside __exit__


When you use the **"with"** statement in Python, it sets up a special context for the code inside it. Here's how it works:

1. Python first evaluates the expression following the **"with"** keyword. This expression should provide a special object known as a **"context guard."**
2. The **"with"** statement then calls the **\_\_enter\_\_** method of the context guard object. This method prepares the necessary setup for the code inside the **"with"** block.
3. Whatever the **\_\_enter\_\_** method returns, Python assigns it to the variable specified after the **"as"** keyword. This allows you to access any data or resources prepared by the **\_\_enter\_\_** method within the block.
4. Python then executes the code inside the **"with"** block.
5. After the block completes execution, regardless of whether there was an error or not, Python calls the **\_\_exit\_\_** method of the context guard object.
6. The **\_\_exit\_\_** method can examine if an exception occurred and decide whether to suppress it or handle it accordingly. If the **\_\_exit\_\_** method returns a true value, it means the exception should be suppressed, allowing the program to continue executing without raising an error.

In [14]:
class HTMLTag:
    def __init__(self, tag_name):
        self.tag_name = tag_name

    def __enter__(self):
        print(f"<{self.tag_name}>", end="")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print(f"</{self.tag_name}>", end="")

In [15]:
with HTMLTag("p"):
    print("some text", end="")

<p>some text</p>

In [16]:
with HTMLTag("p"):
    print("some ", end="")
    with HTMLTag("b"):
        print("bold ", end="")
    print("text", end="")

<p>some <b>bold </b>text</p>

<p>some <b>bold </b>text</p>

## More on `__exit__`

In [17]:
class CtxMng:
    def __init__(self):
        print("inside __init__")

    def __enter__(self):
        print("inside __enter__")

        return (1, 2, 3)

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("inside __exit__")

        if isinstance(exc_value, TypeError):
            print(f"Supressing {exc_value = }")
            return True
        else:
            print(f"Not suppressing {exc_value = }")
            return False

In [18]:
with CtxMng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")

inside __init__
inside __enter__
x = 1, y = 2, z = 3
inside __exit__
Not suppressing exc_value = None


In [19]:
with CtxMng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")
    raise TypeError()

inside __init__
inside __enter__
x = 1, y = 2, z = 3
inside __exit__
Supressing exc_value = TypeError()


In [20]:
with CtxMng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")
    raise ValueError()

inside __init__
inside __enter__
x = 1, y = 2, z = 3
inside __exit__
Not suppressing exc_value = ValueError()


ValueError: 

### Some pattern for context manager

- **Open/Close pattern**
    - Open --> Perform some read/write operation on file --> Close file
    - Open --> Perform some operation on socker --> Close socket connection

```python
with open('example.txt', 'w') as file:
    file.write('Hello, world!')
```

-  **Start/Stop pattern**
    - Start DB transaction --> Perform database operation --> Commit or rollback transaction
    - Start Timer --> Run something --> Stop Timer
    
```python
import sqlite3

conn = sqlite3.connect('mydatabase.db')
with conn:
    cursor = conn.cursor()
    cursor.execute("UPDATE mytable SET value = 42 WHERE id = 1")
```

- **Lock/Release**
    - Acquire thread lock --> Access resource --> Release lock

```python
import threading

lock = threading.Lock()
with lock:
    # Access shared resource safely
    print("Inside locked section")
```

- **Change/Reset**
    - Redirect stdout to file --> Write something to stdout --> Reset stdout to default
    - Temporarily change environment variables --> Execute code --> Reset environment variables
    
```python
import sys

original_stdout = sys.stdout
with open('output.txt', 'w') as file:
    sys.stdout = file
    print("This goes to the file.")
sys.stdout = original_stdout
print("This goes back to the standard output.")
```

## Writing context manager the easy way (using @contextmanager)

In [21]:
class CtxMng:
    def __init__(self):
        print("inside __init__")

    def __enter__(self):
        print("inside __enter__")

        return (1, 2, 3)

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("inside __exit__")

        if isinstance(exc_value, TypeError):
            print(f"Supressing {exc_value = }")
            return True
        else:
            print(f"No suppressing {exc_value = }")
            return False

In [22]:
from contextlib import contextmanager


@contextmanager
def ctx_mng():
    print("before yield")

    try:
        yield (1, 2, 3)
    except TypeError as exc_value:
        print(f"Supressing {exc_value = }")
    except (
        Exception
    ) as exc_value:  # Not needed but trying to show how it works similar to __exit__d
        f"Not suppressing {exc_value = }"
        raise
    finally:
        print("cleanup")

In [23]:
with ctx_mng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")

before yield
x = 1, y = 2, z = 3
cleanup


In [24]:
with ctx_mng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")
    raise TypeError()

before yield
x = 1, y = 2, z = 3
Supressing exc_value = TypeError()
cleanup


In [25]:
with ctx_mng() as (x, y, z):
    print(f"{x = }, {y = }, {z = }")
    raise ValueError()

before yield
x = 1, y = 2, z = 3
cleanup


ValueError: 

Don't forget the paranthesis for context manager which returns/yields multiple values

In [26]:
with CtxMng() as x, y, z:
    print(f"{x = }, {y = }, {z = }")

inside __init__
inside __enter__
inside __exit__
Supressing exc_value = TypeError("'int' object does not support the context manager protocol")


In [27]:
with CtxMng() as x:
    with y:
        with z:
            print(f"{x = }, {y = }, {z = }")

inside __init__
inside __enter__
inside __exit__
Supressing exc_value = TypeError("'int' object does not support the context manager protocol")


## A practical example use case

### Restore directory

In [28]:
from contextlib import contextmanager
import os


@contextmanager
def chdir(new_dir):
    """Change directory then restore."""
    old_dir = os.getcwd()
    os.chdir(str(new_dir))
    try:
        yield os.getcwd()
    finally:
        os.chdir(old_dir)


print(f"Before entering example {os.getcwd() = }")
with chdir("example") as new_dir:
    print(f"After entering example {os.getcwd() = }")
    print("Do something here")

print(f"Outside context manager {os.getcwd() = }")

Before entering example os.getcwd() = 'c:\\Users\\bhmiller\\OneDrive - Intel Corporation\\Documents\\GitHub\\intermediate-python\\content'
After entering example os.getcwd() = 'c:\\Users\\bhmiller\\OneDrive - Intel Corporation\\Documents\\GitHub\\intermediate-python\\content\\example'
Do something here
Outside context manager os.getcwd() = 'c:\\Users\\bhmiller\\OneDrive - Intel Corporation\\Documents\\GitHub\\intermediate-python\\content'


This is part contextlib module from Python 3.11 onwards: [contextlib.chdir](https://docs.python.org/3/library/contextlib.html#contextlib.chdir)

### Redirect stdout

In [29]:
from contextlib import redirect_stdout
from pathlib import Path


static_dir = Path().resolve() / "static"

with open(static_dir / "contextlib_help.txt", "w") as f:
    with redirect_stdout(f):
        print("This is written using contextlib.redirect_stdout")
        help("contextlib")

### Supress know exceptions

In [30]:
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove("file_which_does_nt_exist.tmp")

### Work with tempdir (good for unittest)

NOTE: In case you use `pytest` for testing there is a better way to handle this.

In [31]:
import tempfile

with tempfile.TemporaryDirectory() as temp_dir:
    print(temp_dir)

C:\Users\bhmiller\AppData\Local\Temp\tmpj_sp_5tn


### Calculate time for some set of operations

In [32]:
import time
from contextlib import contextmanager


@contextmanager
def timer():
    start_time = time.time()
    yield
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")


# Usage of the timer context manager
with timer():
    # Code block to measure execution time
    time.sleep(2)

Execution time: 2.0007057189941406 seconds


\[<< [Sequence, Iterator and Generator](./06_sequence_iterators_and_generators.ipynb) | [Index](./00_index.ipynb) | [Building command line interface](./08_argparse.ipynb) >>\]
